# Récupération du classement journée par journée depuis J1

### On vient concat les journée 1 par 1 depuis J1 pour faire un CSV global

In [50]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from pprint import pprint

#### Aller chercher la journée maximum depuis le site du top 14

In [ ]:
# FONCTION AFFICHAGE DES 3 DATAFRAMES

# def fn_affichage_classements(url):
#     response = requests.get(url)
#     # return(response)
#     if response.status_code == 200:
#         # return('test')

#         # je crée une variable soup surn laquelle j'applique BeautifulSoup
#         # avec en paramètre url.text et je découpe avec html.parser
#         soup = BeautifulSoup(response.text, 'html.parser')
#         # return(soup)   

#         # NOM EQUIPES
#         # je recherche les noms de toutes les équipes avec la balise a et la classe affichée sur la page internet
#         list_nom_equipes = soup.find_all('a', {'class': 'base-link base-link--black'})

#         # je crée une boucle insérer les noms des equipes dans une liste
#         list_team = []
#         for ele in list_nom_equipes:
#                 ele = str(ele)
#                 # Appliquer la regex à chaque élément `ele`
#                 match = re.search(r'>([^<]+)<', ele)
#                 if match:  # Vérifier si la regex a trouvé un résultat
#                     list_team.append(match.group(1).strip())
#         # print(list_team)    
        
#         points_equipes = soup.find_all('div', {'class': 'table-line__cell--small'})
#         list_points_team = []
#         for points in points_equipes:
#             # print(points)
#             points = str(points)
#             regex_points = re.search(r'[0-9]+', points)
#             # print(regex_points)
#             # regex_points[0]
#             # print(regex_points[0])
#             list_points_team.append(regex_points[0])

#         all_team = soup.find_all('div', {'class': 'table-line table-line--ranking-full table-line--ranking-scrollable'})
#         list_matchs_joues = []
#         list_matchs_gagnes = []
#         list_matchs_nuls = []
#         list_matchs_perdus = []

#         for team in range(0, 14):
#             for data in range(1, 5):
#                 if data == 1:
#                     points_test = all_team[team].find_all('div', {'class': ''})[data]
#                     # print(re.search(r'[0-9]+', str(points_test))[0])
#                     list_matchs_joues.append(re.search(r'[0-9]+', str(points_test))[0])
#                 elif data == 2:
#                     points_test = all_team[team].find_all('div', {'class': ''})[data]
#                     # print(re.search(r'[0-9]+', str(points_test))[0])
#                     list_matchs_gagnes.append(re.search(r'[0-9]+', str(points_test))[0])
#                 elif data == 3:
#                     points_test = all_team[team].find_all('div', {'class': ''})[data]
#                     # print(re.search(r'[0-9]+', str(points_test))[0])
#                     list_matchs_nuls.append(re.search(r'[0-9]+', str(points_test))[0])
#                 else:
#                     points_test = all_team[team].find_all('div', {'class': ''})[data]
#                     # print(re.search(r'[0-9]+', str(points_test))[0])
#                     list_matchs_perdus.append(re.search(r'[0-9]+', str(points_test))[0])

#         df = pd.DataFrame({
#                 "Equipes": list_team,
#                 "Pts": list_points_team,
#                 "Matchs joués": list_matchs_joues,
#                 "Gagnés": list_matchs_gagnes,
#                 "Nuls": list_matchs_nuls,
#                 "Perdus": list_matchs_perdus,
#                 "Journée":f'J{journee}'
#                 })
        
        
#         if int(max(list_matchs_joues)) <= journee:
        
#             display(int(max(list_matchs_joues)), journee)
#     else:
#         print("Erreur d'url")

In [91]:
dico_id_club = {'ASM Clermont':1,
                'Stade Toulousain':2,
                'Union Bordeaux-Bègles':3,
                'Stade Rochelais':4,
                'LOU Rugby':5,
                'Castres Olympique':6,
                'Aviron Bayonnais':7,
                'USA Perpignan':8,
                'Racing 92':9,
                'Montpellier Hérault Rugby':10,
                'RC Toulon':11,
                'Stade Français Paris':12,
                'RC Vannes':13,
                'Section Paloise':14}

In [94]:
df_global = pd.DataFrame()

for journee in range(1, 26):
    dico_urls = {f"https://top14.lnr.fr/classement/2024-2025/j{journee}?": "Classement général"}
    for url in dico_urls.keys():
        dico_urls[url]
        # print(dico_urls[url])        
        # print(url)
        response = requests.get(url)
    # return(response)
        if response.status_code == 200:
            # return('test')

            # je crée une variable soup surn laquelle j'applique BeautifulSoup
            # avec en paramètre url.text et je découpe avec html.parser
            soup = BeautifulSoup(response.text, 'html.parser')
            # return(soup)   

            # NOM EQUIPES
            # je recherche les noms de toutes les équipes avec la balise a et la classe affichée sur la page internet
            list_nom_equipes = soup.find_all('a', {'class': 'base-link base-link--black'})

            # je crée une boucle insérer les noms des equipes dans une liste
            list_team = []
            for ele in list_nom_equipes:
                    ele = str(ele)
                    # Appliquer la regex à chaque élément `ele`
                    match = re.search(r'>([^<]+)<', ele)
                    if match:  # Vérifier si la regex a trouvé un résultat
                        list_team.append(match.group(1).strip())
            # print(list_team)    
            
            points_equipes = soup.find_all('div', {'class': 'table-line__cell--small'})
            list_points_team = []
            for points in points_equipes:
                # print(points)
                points = str(points)
                regex_points = re.search(r'[0-9]+', points)
                # print(regex_points)
                # regex_points[0]
                # print(regex_points[0])
                list_points_team.append(regex_points[0])

            all_team = soup.find_all('div', {'class': 'table-line table-line--ranking-full table-line--ranking-scrollable'})
            list_matchs_joues = []
            list_matchs_gagnes = []
            list_matchs_nuls = []
            list_matchs_perdus = []

            for team in range(0, 14):
                for data in range(1, 5):
                    if data == 1:
                        points_test = all_team[team].find_all('div', {'class': ''})[data]
                        # print(re.search(r'[0-9]+', str(points_test))[0])
                        list_matchs_joues.append(re.search(r'[0-9]+', str(points_test))[0])
                    elif data == 2:
                        points_test = all_team[team].find_all('div', {'class': ''})[data]
                        # print(re.search(r'[0-9]+', str(points_test))[0])
                        list_matchs_gagnes.append(re.search(r'[0-9]+', str(points_test))[0])
                    elif data == 3:
                        points_test = all_team[team].find_all('div', {'class': ''})[data]
                        # print(re.search(r'[0-9]+', str(points_test))[0])
                        list_matchs_nuls.append(re.search(r'[0-9]+', str(points_test))[0])
                    else:
                        points_test = all_team[team].find_all('div', {'class': ''})[data]
                        # print(re.search(r'[0-9]+', str(points_test))[0])
                        list_matchs_perdus.append(re.search(r'[0-9]+', str(points_test))[0])

            df = pd.DataFrame({
                    "Equipes": list_team,
                    "Pts": list_points_team,
                    "Matchs joués": list_matchs_joues,
                    "Gagnés": list_matchs_gagnes,
                    "Nuls": list_matchs_nuls,
                    "Perdus": list_matchs_perdus,
                    "Journée":f'J{journee}'
                    })
            
        
            if int(max(list_matchs_joues)) == journee:
            
                # display(df)
                df_global = pd.concat([df_global, df])
            else:
                break
            
        else:
            print("Erreur d'url")
            
df_global['ID_equipe'] = df_global['Equipes'].apply(lambda x : dico_id_club[x])

df_global

,Equipes,Pts,Matchs joués,Gagnés,Nuls,Perdus,Journée,ID_equipe
0,ASM Clermont,5,1,1,0,0,J1,1
1,Stade Toulousain,5,1,1,0,0,J1,2
2,Union Bordeaux-Bègles,4,1,1,0,0,J1,3
3,Stade Rochelais,4,1,1,0,0,J1,4
4,LOU Rugby,4,1,1,0,0,J1,5
...,...,...,...,...,...,...,...,...
9,Section Paloise,29,15,6,0,9,J15,14
10,USA Perpignan,28,15,6,0,9,J15,8
11,Racing 92,26,15,5,1,9,J15,9
12,Stade Français Paris,24,15,5,0,10,J15,12


In [101]:
dico_id_club['Stade Rochelais']

4

In [102]:
df_global.loc[df_global['ID_equipe'] == 4]

,Equipes,Pts,Matchs joués,Gagnés,Nuls,Perdus,Journée,ID_equipe
3,Stade Rochelais,4,1,1,0,0,J1,4
9,Stade Rochelais,4,2,1,0,1,J2,4
4,Stade Rochelais,9,3,2,0,1,J3,4
3,Stade Rochelais,13,4,3,0,1,J4,4
1,Stade Rochelais,18,5,4,0,1,J5,4
2,Stade Rochelais,18,6,4,0,2,J6,4
2,Stade Rochelais,22,7,5,0,2,J7,4
3,Stade Rochelais,22,8,5,0,3,J8,4
2,Stade Rochelais,27,9,6,0,3,J9,4
2,Stade Rochelais,28,10,6,0,4,J10,4


In [107]:
df_global.query("ID_equipe == 4")

,Equipes,Pts,Matchs joués,Gagnés,Nuls,Perdus,Journée,ID_equipe
3,Stade Rochelais,4,1,1,0,0,J1,4
9,Stade Rochelais,4,2,1,0,1,J2,4
4,Stade Rochelais,9,3,2,0,1,J3,4
3,Stade Rochelais,13,4,3,0,1,J4,4
1,Stade Rochelais,18,5,4,0,1,J5,4
2,Stade Rochelais,18,6,4,0,2,J6,4
2,Stade Rochelais,22,7,5,0,2,J7,4
3,Stade Rochelais,22,8,5,0,3,J8,4
2,Stade Rochelais,27,9,6,0,3,J9,4
2,Stade Rochelais,28,10,6,0,4,J10,4
